# Get Index ETF Data

In [76]:
from tqdm import tqdm

In [1]:
def get_data(tickers: str = None):
    import yfinance as yf
    import pandas as pd

    df = yf.download(tickers,
                     period='max',
                     interval='1d',
                     # start="2020-12-01",
                     # end="2023-12-31",
                     group_by="ticker",
                     back_adjust=True,
                     progress=False)

    df = df.stack(level=0, future_stack=True).reset_index()
    df.columns = [col.lower() for col in df.columns]
    df = df.dropna()
    df = df[["date", "ticker", "close"]]
    df["date"] = pd.to_datetime(df["date"].dt.date)
    df = df.sort_values(["ticker", "date"], ascending=[True, True])
    df = df.reset_index(drop=True)
    return df

df = get_data(tickers = ['SPY'])
df

,date,ticker,close
0,1993-01-29,SPY,43.937500
1,1993-02-01,SPY,44.250000
2,1993-02-02,SPY,44.343750
3,1993-02-03,SPY,44.812500
4,1993-02-04,SPY,45.000000
...,...,...,...
7998,2024-11-04,SPY,569.809998
7999,2024-11-05,SPY,576.700012
8000,2024-11-06,SPY,591.039978
8001,2024-11-07,SPY,595.609985


### Investment Window (10 Days)

In [2]:
INVESTMENT_PERIOD = 10 # 252
df = df.head(INVESTMENT_PERIOD).copy().sort_values("date")
df

,date,ticker,close
0,1993-01-29,SPY,43.93750
1,1993-02-01,SPY,44.25000
2,1993-02-02,SPY,44.34375
3,1993-02-03,SPY,44.81250
4,1993-02-04,SPY,45.00000
5,1993-02-05,SPY,44.96875
6,1993-02-08,SPY,44.96875
7,1993-02-09,SPY,44.65625
8,1993-02-10,SPY,44.71875
9,1993-02-11,SPY,44.93750


# Calculate Returns and Risks Metrics

### Cumulative Returns  (expanded code)

In [3]:
df.loc[:, 'daily_returns'] = df['close'].pct_change()
df

,date,ticker,close,daily_returns
0,1993-01-29,SPY,43.93750,NaN
1,1993-02-01,SPY,44.25000,0.007112
2,1993-02-02,SPY,44.34375,0.002119
3,1993-02-03,SPY,44.81250,0.010571
4,1993-02-04,SPY,45.00000,0.004184
5,1993-02-05,SPY,44.96875,-0.000694
6,1993-02-08,SPY,44.96875,0.000000
7,1993-02-09,SPY,44.65625,-0.006949
8,1993-02-10,SPY,44.71875,0.001400
9,1993-02-11,SPY,44.93750,0.004892


In [4]:
df['daily_returns_plus_one'] = df['daily_returns'] + 1
df

,date,ticker,close,daily_returns,daily_returns_plus_one
0,1993-01-29,SPY,43.93750,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,1.007112
2,1993-02-02,SPY,44.34375,0.002119,1.002119
3,1993-02-03,SPY,44.81250,0.010571,1.010571
4,1993-02-04,SPY,45.00000,0.004184,1.004184
5,1993-02-05,SPY,44.96875,-0.000694,0.999306
6,1993-02-08,SPY,44.96875,0.000000,1.000000
7,1993-02-09,SPY,44.65625,-0.006949,0.993051
8,1993-02-10,SPY,44.71875,0.001400,1.001400
9,1993-02-11,SPY,44.93750,0.004892,1.004892


In [5]:
df["returns_pls_1_cumprod"] = df['daily_returns_plus_one'].cumprod()
df

,date,ticker,close,daily_returns,daily_returns_plus_one,returns_pls_1_cumprod
0,1993-01-29,SPY,43.93750,NaN,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,1.007112,1.007112
2,1993-02-02,SPY,44.34375,0.002119,1.002119,1.009246
3,1993-02-03,SPY,44.81250,0.010571,1.010571,1.019915
4,1993-02-04,SPY,45.00000,0.004184,1.004184,1.024182
5,1993-02-05,SPY,44.96875,-0.000694,0.999306,1.023471
6,1993-02-08,SPY,44.96875,0.000000,1.000000,1.023471
7,1993-02-09,SPY,44.65625,-0.006949,0.993051,1.016358
8,1993-02-10,SPY,44.71875,0.001400,1.001400,1.017781
9,1993-02-11,SPY,44.93750,0.004892,1.004892,1.022760


In [6]:
df["cum_returns"] = df['returns_pls_1_cumprod'] - 1
df

,date,ticker,close,daily_returns,daily_returns_plus_one,returns_pls_1_cumprod,cum_returns
0,1993-01-29,SPY,43.93750,NaN,NaN,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,1.007112,1.007112,0.007112
2,1993-02-02,SPY,44.34375,0.002119,1.002119,1.009246,0.009246
3,1993-02-03,SPY,44.81250,0.010571,1.010571,1.019915,0.019915
4,1993-02-04,SPY,45.00000,0.004184,1.004184,1.024182,0.024182
5,1993-02-05,SPY,44.96875,-0.000694,0.999306,1.023471,0.023471
6,1993-02-08,SPY,44.96875,0.000000,1.000000,1.023471,0.023471
7,1993-02-09,SPY,44.65625,-0.006949,0.993051,1.016358,0.016358
8,1993-02-10,SPY,44.71875,0.001400,1.001400,1.017781,0.017781
9,1993-02-11,SPY,44.93750,0.004892,1.004892,1.022760,0.022760


### Cumulative Returns  (1 liner)

In [7]:
df = get_data(tickers = ['SPY'])
df = df.head(10)

,date,ticker,close
0,1993-01-29,SPY,43.93750
1,1993-02-01,SPY,44.25000
2,1993-02-02,SPY,44.34375
3,1993-02-03,SPY,44.81250
4,1993-02-04,SPY,45.00000
5,1993-02-05,SPY,44.96875
6,1993-02-08,SPY,44.96875
7,1993-02-09,SPY,44.65625
8,1993-02-10,SPY,44.71875
9,1993-02-11,SPY,44.93750


In [8]:
df["cum_returns"] = ( 1 + df['close'].pct_change() ).cumprod() - 1
df

,date,ticker,close,cum_returns
0,1993-01-29,SPY,43.93750,NaN
1,1993-02-01,SPY,44.25000,0.007112
2,1993-02-02,SPY,44.34375,0.009246
3,1993-02-03,SPY,44.81250,0.019915
4,1993-02-04,SPY,45.00000,0.024182
5,1993-02-05,SPY,44.96875,0.023471
6,1993-02-08,SPY,44.96875,0.023471
7,1993-02-09,SPY,44.65625,0.016358
8,1993-02-10,SPY,44.71875,0.017781
9,1993-02-11,SPY,44.93750,0.022760


### Maximum Drawdown (expanded)

In [24]:
df = get_data(tickers = ['SPY'])
df = df.head(10)

### Cumulative Max

In [25]:
df['rolling_max'] = df['close'].cummax()
df

,date,ticker,close,rolling_max
0,1993-01-29,SPY,43.93750,43.93750
1,1993-02-01,SPY,44.25000,44.25000
2,1993-02-02,SPY,44.34375,44.34375
3,1993-02-03,SPY,44.81250,44.81250
4,1993-02-04,SPY,45.00000,45.00000
5,1993-02-05,SPY,44.96875,45.00000
6,1993-02-08,SPY,44.96875,45.00000
7,1993-02-09,SPY,44.65625,45.00000
8,1993-02-10,SPY,44.71875,45.00000
9,1993-02-11,SPY,44.93750,45.00000


### Close - Max

In [26]:
df['close_max_diff'] = df['close'] - df['rolling_max']
df

,date,ticker,close,rolling_max,close_max_diff
0,1993-01-29,SPY,43.93750,43.93750,0.00000
1,1993-02-01,SPY,44.25000,44.25000,0.00000
2,1993-02-02,SPY,44.34375,44.34375,0.00000
3,1993-02-03,SPY,44.81250,44.81250,0.00000
4,1993-02-04,SPY,45.00000,45.00000,0.00000
5,1993-02-05,SPY,44.96875,45.00000,-0.03125
6,1993-02-08,SPY,44.96875,45.00000,-0.03125
7,1993-02-09,SPY,44.65625,45.00000,-0.34375
8,1993-02-10,SPY,44.71875,45.00000,-0.28125
9,1993-02-11,SPY,44.93750,45.00000,-0.06250


### Drawdown

In [27]:
df['drawdown'] = df['close_max_diff'] / df['rolling_max']
df

,date,ticker,close,rolling_max,close_max_diff,drawdown
0,1993-01-29,SPY,43.93750,43.93750,0.00000,0.000000
1,1993-02-01,SPY,44.25000,44.25000,0.00000,0.000000
2,1993-02-02,SPY,44.34375,44.34375,0.00000,0.000000
3,1993-02-03,SPY,44.81250,44.81250,0.00000,0.000000
4,1993-02-04,SPY,45.00000,45.00000,0.00000,0.000000
5,1993-02-05,SPY,44.96875,45.00000,-0.03125,-0.000694
6,1993-02-08,SPY,44.96875,45.00000,-0.03125,-0.000694
7,1993-02-09,SPY,44.65625,45.00000,-0.34375,-0.007639
8,1993-02-10,SPY,44.71875,45.00000,-0.28125,-0.006250
9,1993-02-11,SPY,44.93750,45.00000,-0.06250,-0.001389


### Max Drawdown

In [28]:
df['rolling_max_drawdown'] = df['drawdown'].cummin()
df

,date,ticker,close,rolling_max,close_max_diff,drawdown,rolling_max_drawdown
0,1993-01-29,SPY,43.93750,43.93750,0.00000,0.000000,0.000000
1,1993-02-01,SPY,44.25000,44.25000,0.00000,0.000000,0.000000
2,1993-02-02,SPY,44.34375,44.34375,0.00000,0.000000,0.000000
3,1993-02-03,SPY,44.81250,44.81250,0.00000,0.000000,0.000000
4,1993-02-04,SPY,45.00000,45.00000,0.00000,0.000000,0.000000
5,1993-02-05,SPY,44.96875,45.00000,-0.03125,-0.000694,-0.000694
6,1993-02-08,SPY,44.96875,45.00000,-0.03125,-0.000694,-0.000694
7,1993-02-09,SPY,44.65625,45.00000,-0.34375,-0.007639,-0.007639
8,1993-02-10,SPY,44.71875,45.00000,-0.28125,-0.006250,-0.007639
9,1993-02-11,SPY,44.93750,45.00000,-0.06250,-0.001389,-0.007639


### Maximum Drawdown (1-liner)

In [33]:
# rolling max
df['rolling_max'] = df['close'].cummax()
# close - max
df['close_max_diff'] = df['close'] - df['rolling_max']
# drawdown
df['drawdown'] = df['close_max_diff'] / df['rolling_max']
# max drawdown
df['rolling_max_drawdown'] = df['drawdown'].cummin()
df

,date,ticker,close,rolling_max,close_max_diff,drawdown,rolling_max_drawdown
0,1993-01-29,SPY,43.93750,43.93750,0.00000,0.000000,0.000000
1,1993-02-01,SPY,44.25000,44.25000,0.00000,0.000000,0.000000
2,1993-02-02,SPY,44.34375,44.34375,0.00000,0.000000,0.000000
3,1993-02-03,SPY,44.81250,44.81250,0.00000,0.000000,0.000000
4,1993-02-04,SPY,45.00000,45.00000,0.00000,0.000000,0.000000
5,1993-02-05,SPY,44.96875,45.00000,-0.03125,-0.000694,-0.000694
6,1993-02-08,SPY,44.96875,45.00000,-0.03125,-0.000694,-0.000694
7,1993-02-09,SPY,44.65625,45.00000,-0.34375,-0.007639,-0.007639
8,1993-02-10,SPY,44.71875,45.00000,-0.28125,-0.006250,-0.007639
9,1993-02-11,SPY,44.93750,45.00000,-0.06250,-0.001389,-0.007639


In [41]:
df = get_data(tickers = ['SPY'])
df = df.head(10)

In [42]:
# rolling max
# df['rolling_max'] = (df['close'].cummax())
# close - max
# df['close_max_diff'] = (df['close'] - df['rolling_max'])
# drawdown
# df['drawdown'] = (df['close_max_diff'] / df['rolling_max'])
# max drawdown
df['rolling_max_drawdown'] = ((df['close'] - (df['close'].cummax())) / df['close'].cummax()).cummin()
df

,date,ticker,close,rolling_max_drawdown
0,1993-01-29,SPY,43.93750,0.000000
1,1993-02-01,SPY,44.25000,0.000000
2,1993-02-02,SPY,44.34375,0.000000
3,1993-02-03,SPY,44.81250,0.000000
4,1993-02-04,SPY,45.00000,0.000000
5,1993-02-05,SPY,44.96875,-0.000694
6,1993-02-08,SPY,44.96875,-0.000694
7,1993-02-09,SPY,44.65625,-0.007639
8,1993-02-10,SPY,44.71875,-0.007639
9,1993-02-11,SPY,44.93750,-0.007639


### Sharpe Ratio

In [48]:
df = get_data(tickers = ['SPY'])
df = df.head(10)

,date,ticker,close
0,1993-01-29,SPY,43.93750
1,1993-02-01,SPY,44.25000
2,1993-02-02,SPY,44.34375
3,1993-02-03,SPY,44.81250
4,1993-02-04,SPY,45.00000
5,1993-02-05,SPY,44.96875
6,1993-02-08,SPY,44.96875
7,1993-02-09,SPY,44.65625
8,1993-02-10,SPY,44.71875
9,1993-02-11,SPY,44.93750


### Rolling Average Returns

In [49]:
df['daily_returns'] = df['close'].pct_change()
df['returns_rolling_mean'] = df['daily_returns'].expanding().mean()
df


,date,ticker,close,daily_returns,returns_rolling_mean
0,1993-01-29,SPY,43.93750,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,0.007112
2,1993-02-02,SPY,44.34375,0.002119,0.004616
3,1993-02-03,SPY,44.81250,0.010571,0.006601
4,1993-02-04,SPY,45.00000,0.004184,0.005996
5,1993-02-05,SPY,44.96875,-0.000694,0.004658
6,1993-02-08,SPY,44.96875,0.000000,0.003882
7,1993-02-09,SPY,44.65625,-0.006949,0.002335
8,1993-02-10,SPY,44.71875,0.001400,0.002218
9,1993-02-11,SPY,44.93750,0.004892,0.002515


### Rolling Standard Deviation

In [50]:
df['returns_rolling_std'] = df['daily_returns'].expanding().std()
df


,date,ticker,close,daily_returns,returns_rolling_mean,returns_rolling_std
0,1993-01-29,SPY,43.93750,NaN,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,0.007112,NaN
2,1993-02-02,SPY,44.34375,0.002119,0.004616,0.003531
3,1993-02-03,SPY,44.81250,0.010571,0.006601,0.004249
4,1993-02-04,SPY,45.00000,0.004184,0.005996,0.003674
5,1993-02-05,SPY,44.96875,-0.000694,0.004658,0.004368
6,1993-02-08,SPY,44.96875,0.000000,0.003882,0.004345
7,1993-02-09,SPY,44.65625,-0.006949,0.002335,0.005700
8,1993-02-10,SPY,44.71875,0.001400,0.002218,0.005288
9,1993-02-11,SPY,44.93750,0.004892,0.002515,0.005026


### Sharpe Ratio

In [51]:
df["sharpe"] = df['returns_rolling_mean'] / df['returns_rolling_std']
df

,date,ticker,close,daily_returns,returns_rolling_mean,returns_rolling_std,sharpe
0,1993-01-29,SPY,43.93750,NaN,NaN,NaN,NaN
1,1993-02-01,SPY,44.25000,0.007112,0.007112,NaN,NaN
2,1993-02-02,SPY,44.34375,0.002119,0.004616,0.003531,1.307102
3,1993-02-03,SPY,44.81250,0.010571,0.006601,0.004249,1.553354
4,1993-02-04,SPY,45.00000,0.004184,0.005996,0.003674,1.632194
5,1993-02-05,SPY,44.96875,-0.000694,0.004658,0.004368,1.066535
6,1993-02-08,SPY,44.96875,0.000000,0.003882,0.004345,0.893445
7,1993-02-09,SPY,44.65625,-0.006949,0.002335,0.005700,0.409574
8,1993-02-10,SPY,44.71875,0.001400,0.002218,0.005288,0.419421
9,1993-02-11,SPY,44.93750,0.004892,0.002515,0.005026,0.500389


### Sharpe Ratio (1-liner)

In [104]:
# 10 year
0.0431

0.0431

In [109]:
daily_risk_free_rate = 0.000016745000791186

((1 + daily_risk_free_rate) ** (252*10)) - 1

0.043100000000476646

In [110]:
df = get_data(tickers = ['SPY'])
df = df.head(10)

In [113]:
df['sharpe'] = ( 
                    df['close'].pct_change() - daily_risk_free_rate 
               ).expanding().mean() / df['close'].pct_change().expanding().std()
df

,date,ticker,close,sharpe
0,1993-01-29,SPY,43.93750,NaN
1,1993-02-01,SPY,44.25000,NaN
2,1993-02-02,SPY,44.34375,1.302360
3,1993-02-03,SPY,44.81250,1.549413
4,1993-02-04,SPY,45.00000,1.627637
5,1993-02-05,SPY,44.96875,1.062701
6,1993-02-08,SPY,44.96875,0.889591
7,1993-02-09,SPY,44.65625,0.406636
8,1993-02-10,SPY,44.71875,0.416254
9,1993-02-11,SPY,44.93750,0.497057


# Quants Combined

In [120]:
def calc_backtest(df):
    
    # roi
    df["cumulative_returns"] = ( 1 + df['close'].pct_change() ).cumprod() - 1
    
    # mdd
    df['rolling_max_drawdown'] = ((df['close'] - (df['close'].cummax())) / df['close'].cummax()).cummin()
    
    # sharpe
    df['cumulative_sharpe'] = ( df['close'].pct_change() - daily_risk_free_rate  ).expanding().mean() / df['close'].pct_change().expanding().std()

    return df

In [121]:
df = get_data(tickers = ['SPY'])
df = df.head(10)
df = calc_backtest(df)
df

,date,ticker,close,cumulative_returns,rolling_max_drawdown,cumulative_sharpe
0,1993-01-29,SPY,43.93750,NaN,0.000000,NaN
1,1993-02-01,SPY,44.25000,0.007112,0.000000,NaN
2,1993-02-02,SPY,44.34375,0.009246,0.000000,1.302360
3,1993-02-03,SPY,44.81250,0.019915,0.000000,1.549413
4,1993-02-04,SPY,45.00000,0.024182,0.000000,1.627637
5,1993-02-05,SPY,44.96875,0.023471,-0.000694,1.062701
6,1993-02-08,SPY,44.96875,0.023471,-0.000694,0.889591
7,1993-02-09,SPY,44.65625,0.016358,-0.007639,0.406636
8,1993-02-10,SPY,44.71875,0.017781,-0.007639,0.416254
9,1993-02-11,SPY,44.93750,0.022760,-0.007639,0.497057


# Sliding Window

In [122]:
TRADING_YR = 252
window_addend = TRANDING_YR // 2
TRANDING_YR, window_addend

(252, 126)

In [123]:
df = get_data(tickers = ['SPY'])
df.shape

(8003, 3)

In [124]:
investment_winow_sizes = list(range(TRADING_YR, len(df), window_addend))
investment_winow_sizes[0:3], investment_winow_sizes[-4:-1]

([252, 378, 504], [7560, 7686, 7812])

In [125]:
backtest_results = []

# loop over various investment periods 
for investment_winow_size in tqdm(investment_winow_sizes):
    
    # loop over day ranges in entire history
    for i in range(0, len(df)-investment_winow_size + 1):
        
        start_idx = i
        end_idx = i + investment_winow_size                        
                    
        window_df = df.iloc[start_idx: end_idx].copy()
        window_df = calc_backtest(window_df)
        
        backtest_results.append({
            "holding_days": investment_winow_size,
            "start_date": window_df['date'].iloc[0],
            "end_date": window_df['date'].iloc[-1],
            "roi": window_df['rolling_max_drawdown'].iloc[-1],
            "sharpe": window_df['cumulative_sharpe'].iloc[-1],
            "mdd": window_df['rolling_max_drawdown'].min()
        })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [10:41<00:00, 10.35s/it]


In [126]:
import pandas as pd
pd.DataFrame(backtest_results)

,holding_days,start_date,end_date,roi,sharpe,mdd
0,252,1993-01-29,1994-01-26,-0.051230,0.051368,-0.051230
1,252,1993-02-01,1994-01-27,-0.051230,0.052744,-0.051230
2,252,1993-02-02,1994-01-28,-0.051230,0.053083,-0.051230
3,252,1993-02-03,1994-01-31,-0.051230,0.050911,-0.051230
4,252,1993-02-04,1994-02-01,-0.051230,0.044352,-0.051230
...,...,...,...,...,...,...
242353,7938,1993-04-28,2024-11-04,-0.564737,0.031914,-0.564737
242354,7938,1993-04-29,2024-11-05,-0.564737,0.031995,-0.564737
242355,7938,1993-04-30,2024-11-06,-0.564737,0.032237,-0.564737
242356,7938,1993-05-03,2024-11-07,-0.564737,0.032251,-0.564737
